In [ ]:
import sys
sys.path.append("../")
import os
print(os.getcwd())
%pylab inline
import GPyOpt
import GPy
from AlphaOpt import components

In [ ]:
from GPyOpt.objective_examples.experiments1d import *
from GPyOpt.objective_examples.experiments2d import *

# Output
func  = forrester()
y = forrester()
input_dim = 1
objective = GPyOpt.core.task.SingleObjective(lambda x: y.f(x))

# Input
v1_dom = func.bounds[0]
#v2_dom = func.bounds[1]

space = GPyOpt.Design_space(space =[{'name': 'var_1', 'type': 'continuous', 'domain': v1_dom}])
                                   #{'name': 'var_2', 'type': 'continuous', 'domain': v2_dom}])

In [ ]:
# Bayesian Optimization Components
# GP models
kernel = GPy.kern.Matern52(input_dim=input_dim, ARD=True)
model = GPyOpt.models.GPModel(kernel=kernel, optimize_restarts=10,verbose=False)

# Cost model
# cost = GPyOpt.core.task.CostModel('evaluation_time')

In [ ]:
from GPyOpt.core.evaluators.base import EvaluatorBase
class MultiAcquisitions(EvaluatorBase):
    def __init__(self, *args):
        self.acquisitions = args

    def compute_batch(self):
        X_batch = self.acquisitions[0].optimize()
        for i in range(1, len(self.acquisitions)):
            X_batch = np.vstack((X_batch,self.acquisitions[i].optimize()))
        return X_batch

In [ ]:
# Decision models
acq_opt = GPyOpt.optimization.AcquisitionOptimizer(space)

# Use EI as the baseline
acq_fxs = [GPyOpt.acquisitions.AcquisitionEI(model, space, acq_opt), 
          components.EIXplore(model, space, acq_opt),
          GPyOpt.acquisitions.AcquisitionMPI(model, space, acq_opt),
          GPyOpt.acquisitions.AcquisitionLCB(model, space, acq_opt)]


eval_fx = MultiAcquisitions(*acq_fxs)

In [ ]:
from GPyOpt.methods.modular_bayesian_optimization import ModularBayesianOptimization
from GPyOpt.core.task.objective import SingleObjective
class CustomBO(ModularBayesianOptimization):
    def __init__(self, model, space, objective, acquisition, evaluator, X_init, Y_init=None, cost = None, normalize_Y = True, model_update_interval = 1, num_cores=4):
        super().__init__(model                 = model, 
                        space                  = space, 
                        objective              = objective, 
                        acquisition            = acquisition, 
                        evaluator              = evaluator,
                        X_init                 = X_init, 
                        Y_init                 = Y_init,
                        cost                   = cost,
                        normalize_Y            = normalize_Y, 
                        model_update_interval  = model_update_interval)
        self.num_cores = num_cores
        self.objective = SingleObjective(self.objective, num_cores=self.num_cores)
        self.run_optimization(0)


In [ ]:
X_init = GPyOpt.util.stats.initial_design('random', space, 2)
num_cores = 4
bo = CustomBO(model=model,objective=objective,space=space,acquisition=None,X_init=X_init,normalize_Y = True,evaluator = eval_fx)

In [ ]:
bo.run_optimization(max_iter = 10, eps=1e-10, verbosity=True)